In [24]:
!pip install huggingface-hub==0.26.2 transformers==4.46.2 peft==0.13.2 accelerate==1.1.1 trl==0.12.1 bitsandbytes==0.45.2 datasets==3.1.0 safetensors==0.4.5 pandas==2.2.2 matplotlib==3.8.0 numpy==1.26.4

In [25]:
import os
import torch
from datasets import load_dataset
from peft import get_peft_model, LoraConfig, prepare_model_for_kbit_training
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from trl import SFTConfig, SFTTrainer

In [26]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
repo_id = 'microsoft/Phi-3-mini-4k-instruct'
model = AutoModelForCausalLM.from_pretrained(repo_id, device_map='cuda:0', quantization_config=bnb_config)

config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/16.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [ ]:
print(model.get_memory_footprint()/1000000)

2206.347264


In [ ]:
model

Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3SdpaAttention(
          (o_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear4bit(in_features=3072, out_features=9216, bias=False)
          (rotary_emb): Phi3RotaryEmbedding()
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear4bit(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
        (post_attention_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
      )
    )
    (norm): Phi3RMSNorm((3072

In [ ]:
model = prepare_model_for_kbit_training(model)

config = LoraConfig(
    r=8, # Entre mas pequeño menos parametros necesitara entrenar.
    lora_alpha=16, # multiplicador, normalmente debe ser 2*r
    bias='none',
    lora_dropout=0.05,
    task_type='CAUSAL_LM',
    # Los modelos mas nuevos debemos especificarle que modulos queremos apuntar
    # en el fine tunning.
    target_modules=['o_proj', 'qkv_proj', 'gate_up_proj', 'down_proj']
)

model = get_peft_model(model, config)
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Phi3ForCausalLM(
      (model): Phi3Model(
        (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
        (embed_dropout): Dropout(p=0.0, inplace=False)
        (layers): ModuleList(
          (0-31): 32 x Phi3DecoderLayer(
            (self_attn): Phi3SdpaAttention(
              (o_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magn

In [ ]:
print(model.get_memory_footprint()/1e6)

2651.080704


In [ ]:
train_p, tot_p = model.get_nb_trainable_parameters()
print(f'Parametros entrenables:      {train_p/1e6:.2f}M')
print(f'Parametros totales:          {tot_p/1e6:.2f}M')
print(f'% de parametros entrenables: {100*train_p/tot_p:.2f}%')

Parametros entrenables:      12.58M
Parametros totales:          3833.66M
% de parametros entrenables: 0.33%


In [ ]:
#dataset = load_dataset("medalpaca/medical_meadow_wikidoc", split='train')
dataset = load_dataset("dvgodoy/yoda_sentences", split='train')
dataset

README.md:   0%|          | 0.00/531 [00:00<?, ?B/s]

sentences.csv:   0%|          | 0.00/98.4k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/720 [00:00<?, ? examples/s]

Dataset({
    features: ['sentence', 'translation', 'translation_extra'],
    num_rows: 720
})

In [ ]:
#dataset = dataset.rename_column("input", "prompt")
#dataset = dataset.rename_column("output", "completion")
#dataset = dataset.remove_columns(["instruction"])
dataset = dataset.rename_column("sentence", "prompt")
dataset = dataset.rename_column("translation_extra", "completion")
dataset = dataset.remove_columns(["translation"])
dataset

Dataset({
    features: ['prompt', 'completion'],
    num_rows: 720
})

In [ ]:
dataset[0]

{'prompt': 'The birch canoe slid on the smooth planks.',
 'completion': 'On the smooth planks, the birch canoe slid. Yes, hrrrm.'}

In [ ]:
def format_dataset(examples):
  converted_sample = [
      {"role": "user", "content": examples["prompt"]},
      {"role": "assistant", "content": examples["completion"]},
  ]
  return {'messages': converted_sample}

In [ ]:
dataset = dataset.map(format_dataset).remove_columns(['prompt', 'completion'])
dataset[0]['messages']

Map:   0%|          | 0/720 [00:00<?, ? examples/s]

[{'content': 'The birch canoe slid on the smooth planks.', 'role': 'user'},
 {'content': 'On the smooth planks, the birch canoe slid. Yes, hrrrm.',
  'role': 'assistant'}]

In [ ]:
#split = dataset.train_test_split(test_size=0.9)
train = dataset

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(repo_id)
print(tokenizer.chat_template)

tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

{% for message in messages %}{% if message['role'] == 'system' %}{{'<|system|>
' + message['content'] + '<|end|>
'}}{% elif message['role'] == 'user' %}{{'<|user|>
' + message['content'] + '<|end|>
'}}{% elif message['role'] == 'assistant' %}{{'<|assistant|>
' + message['content'] + '<|end|>
'}}{% endif %}{% endfor %}{% if add_generation_prompt %}{{ '<|assistant|>
' }}{% else %}{{ eos_token }}{% endif %}


In [ ]:
print(tokenizer.apply_chat_template(conversation=train['messages'][32], tokenize=False))

<|user|>
Note closely the size of the gas tank.<|end|>
<|assistant|>
Closely, note the size of the gas tank, you must.<|end|>
<|endoftext|>


In [ ]:
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id = tokenizer.unk_token_id

In [ ]:
sft = SFTConfig(
    gradient_checkpointing=True, # Con esto ahorramos un montón de memoria
    gradient_checkpointing_kwargs={"use_reentrant": False}, # Con las nuevas version de PyTorch necesitamos poner este argumento, mas información acá https://github.com/huggingface/transformers/issues/28536
    # Acumulacion del gradient y batch size
    gradient_accumulation_steps=1,
    per_device_train_batch_size=16,
    auto_find_batch_size=True, # Si el batch size que usas puede ocasionar un OOM (Out of Memory) lo dividimos en 2 hasta que funcione.

    #max_seq_length=768, # El tamaño de nuestra ventana de contexto.
    max_seq_length=64,
    packing=True,

    num_train_epochs=10,
    learning_rate=3e-4,

    optim='paged_adamw_8bit',

    logging_steps=10,
    logging_dir='./logs',
    output_dir='./phi3-mini-med-adapter',
    report_to='none'
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    processing_class=tokenizer,
    args=sft,
    #train_dataset=split['train']
    train_dataset=train
)

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:403: UserWarning: You passed a processing_class with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `processing_class.padding_side = 'right'` to your code.
  warnings.warn(


In [ ]:
dl = trainer.get_train_dataloader()
batch = next(iter(dl))

In [ ]:
len(batch['input_ids'][0]), len(batch['labels'][0])

(64, 64)

In [23]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss
10,3.060000
20,1.758000
30,1.552500
40,1.429000
50,1.349400
60,1.243600
70,1.144000
80,0.925300
90,0.867200
100,0.593900


Step,Training Loss
10,3.060000
20,1.758000
30,1.552500
40,1.429000
50,1.349400
60,1.243600
70,1.144000
80,0.925300
90,0.867200
100,0.593900


TrainOutput(global_step=220, training_loss=0.8187831163406372, metrics={'train_runtime': 1602.4913, 'train_samples_per_second': 2.19, 'train_steps_per_second': 0.137, 'total_flos': 5034400555991040.0, 'train_loss': 0.8187831163406372, 'epoch': 10.0})

In [27]:
def encode_prompt(tokenizer, sentence):
  sample = [{'role': 'user', 'content': sentence}]
  prompt = tokenizer.apply_chat_template(conversation=sample, tokenize=False, add_generation_prompt=True)
  return prompt

In [28]:
#sentence = 'What are the historical background and symptoms of Candida-induced vulvovaginitis?'
sentence = 'The Force is strong in you!'
prompt = encode_prompt(tokenizer, sentence)
print(prompt)

<|user|>
The Force is strong in you!<|end|>
<|assistant|>



In [30]:
def inference(model, tokenizer, prompt, max_new_tokens=64, skip_special_tokens=False):
  tokenized_input = tokenizer(
      prompt, return_tensors='pt', add_special_tokens=False
  ).to(model.device)

  model.eval()

  gen_output = model.generate(**tokenized_input,
                              eos_token_id=tokenizer.eos_token_id,
                              max_new_tokens=max_new_tokens)

  output = tokenizer.batch_decode(gen_output, skip_special_tokens=skip_special_tokens)

  return output[0]

In [33]:
print(inference(model, tokenizer, prompt))

<|user|> The Force is strong in you!<|end|><|assistant|> Strong in you, the Force is!<|end|><|endoftext|>


In [34]:
trainer.save_model('local-phi3-mini-yoda-adapter')

In [35]:
!pip install huggingface_hub

In [36]:
from huggingface_hub import login
login()

In [37]:
trainer.push_to_hub()

adapter_model.safetensors:   0%|          | 0.00/50.4M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/srmorfi/phi3-mini-med-adapter/commit/39a9d356f23085373b45d8c6efec56fa9ad1641c', commit_message='End of training', commit_description='', oid='39a9d356f23085373b45d8c6efec56fa9ad1641c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/srmorfi/phi3-mini-med-adapter', endpoint='https://huggingface.co', repo_type='model', repo_id='srmorfi/phi3-mini-med-adapter'), pr_revision=None, pr_num=None)

[REGRESAR](../02_LoRa_para_fine_tuning_eficiente_de_modelos_gigantes.md)